# Updating Training data

## Requierement

- An orthoimage from the Micasense RedEdge Dual MX multispectral camera, coded as a 16-bit integer (values ranging from 0 to 10000)
- A shapefile of the new training data, containing at least one column named "True_Class", which holds the name of the class of the polygon.

In [53]:
from osgeo import gdal, ogr
from fastbook import *
from math import floor




### Data opening

In [43]:
training_raster = "Belon_1104_MS"
training_shp = "../Data/shp/Training_Belon_1104/Training_Belon_1104.shp"

In [49]:
old_training_data = pd.read_csv("../Data/Training/DISCOV_BiCOME_Training_reframed.csv", low_memory=False,sep=";")

training_rast = gdal.Open('../Data/img/' + training_raster + '.tif')
num_bands = training_rast.RasterCount

gt_forward = training_rast.GetGeoTransform()
gt_reverse = gdal.InvGeoTransform(gt_forward)

rb = []  # A list to hold all the raster bands

for band_num in range(1, 10):
    rb.append(training_rast.GetRasterBand(band_num))

shapefile = ogr.Open(training_shp)
layer = shapefile.GetLayer()

pixel_values = [] 

In [ ]:
for feat in layer:
    fid = feat.GetFID()  # Get the feature ID
    geom = feat.GetGeometryRef()
    mx, my = geom.GetX(), geom.GetY()  # Coordinates in map units

    # Convert from map to pixel coordinates
    px, py = gdal.ApplyGeoTransform(gt_reverse, mx, my)
    px = floor(px)  # X pixel
    py = floor(py)  # Y pixel

    # Read pixel values for all bands and store in a list along with feature ID
    pixel_values_for_feature = [fid]
    for band_num in range(1,10):
        intval = rb[band_num - 1].ReadAsArray(px, py, 1, 1)
        pixel_values_for_feature.append(intval[0][0])

    pixel_values.append(pixel_values_for_feature)

# Print the resulting array
for row in pixel_values:
    print(row)


range(1, 11)

In [60]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping


In [61]:


shapefile = gpd.read_file(training_shp)
# extract the geometry in GeoJSON format
geoms = shapefile.geometry.values # list of shapely geometries
geometry = geoms[0] # shapely geometry
# transform to GeJSON format

geoms = [mapping(geoms[0])]
# extract the raster values values within the polygon 
with rasterio.open('../Data/img/' + training_raster + '.tif') as src:
     out_image, out_transform = mask(src, geoms, crop=True)

In [67]:
data = out_image.data
data